In [1]:
import pandas as pd
import random

from functools import partial
import math
import os
import tqdm
import numpy as np
#import bamboolib as bam
import sklearn
from matplotlib import pyplot 
import seaborn as sns
from sklearn import metrics
from matplotlib import pyplot as plt
    
from kneed import KneeLocator
import scipy
    

# Chargement et préparation des données

In [2]:
aymeric = "/home/aymeric/python-scripts/anses_medialab/datas" #aymeric
jp = 'd:/Projects/Medialab/Anses/' # Jean Philippe

path= aymeric
tfidf='tfidf'
#domain='media'
#domain='twitter'
#domain='facebook'

index_file_JP="indexation_results/index_alldefault_nameexhaustive_aymeric.csv"
index_file_AL=f"index_alldefault_nameexhaustive_aymeric.csv"
index_file=f"{path}/{index_file_AL}"
index_file

'/home/aymeric/python-scripts/anses_medialab/datas/index_alldefault_nameexhaustive_aymeric.csv'

In [3]:
df=pd.read_csv(index_file, sep='\t', dtype={"doc_id" :"string", "sent_id": "string"})


In [4]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['yearmonth']=(df['date'].dt.strftime('%Y-%m'))
df=df[~df['yearmonth'].isin(['2008-04', '2010-01','2021-02'])]
df=df.dropna(subset=['yearmonth'])
len(df)

7832372

## Fréquence des termes par documents (sur facebook)

In [6]:
df["doc_sent_id"] = df["doc_id"]+"_"+df["sent_id"]

df_fb = df.loc[df["origin"] == "facebook"]
N=500
topN=df_fb["term"].value_counts().index[:N]
df=df_fb[df_fb["term"].isin(topN)]

df_term_freq = df_fb.groupby(['term', 'doc_id']).agg(freq=('sent_id', 'size')).reset_index()

,term,doc_id,freq
0,% des échantillons,102163,1
1,% des échantillons,102406,1
2,% des échantillons,102422,1
3,% des échantillons,10722,1
4,% des échantillons,112869,1
...,...,...,...
2284289,œuf,99387,1
2284290,œuf,99394,1
2284291,œuf,99464,1
2284292,œuf,99496,3


## pivotement du dataframe pour produire une matrice documents-termes

In [8]:
table = pd.pivot_table(df_term_freq, values='freq', index=['doc_id'],columns=["term"], aggfunc=np.sum, fill_value=0)

## Création de la matrice termes-termes
multiplication de la matrice documents-termes par sa transposée

In [9]:
term_term = np.dot(table.T, table) # multiplication de la transposé de

In [13]:
mi_matrix = []
type(mi_matrix)
mi_matrix

[]

In [15]:
freq_term = [x for x in term_term.diagonal()] # pour obtenir la fréquence de chaque terme (somme de chaque ligne)

#Nombre de paires est égale à la somme des lignes moins la diagonale (à diviser par 2?)
Nb_pairs = sum(np.sum(term_term,axis=1).tolist()) -  sum(freq_term) 

for i in range(len(term_term)):
    list_mi = []
    for j in range(len(term_term)):
        nb_cooc = term_term[i,j] #nombre de cooccurrences des termes i et j
        freq_i = freq_term[i] # fréquence de i
        freq_j = freq_term[j] # fréquence de j
        
        if i == j : 
            mutual_info = 0
        else:
            if nb_cooc == 0 :
                mutual_info = 0
            else: #calcule mutual information
                num = nb_cooc*Nb_pairs
                denom = freq_i*freq_j
                mutual_info = np.log(num/denom)
            
        list_mi.append(mutual_info)
    mi_matrix.append(list_mi)



## Calcule du cosine

In [31]:
mi_array = np.array(mi_matrix)
m =scipy.sparse.csc_matrix(mi_array)
dist=metrics.pairwise.cosine_similarity(m)
dist

array([[1.        , 0.42023496, 0.63973208, ..., 0.70165638, 0.75650301,
        0.65661498],
       [0.42023496, 1.        , 0.4138105 , ..., 0.47898796, 0.42830392,
        0.39266756],
       [0.63973208, 0.4138105 , 1.        , ..., 0.82454437, 0.86006469,
        0.7783489 ],
       ...,
       [0.70165638, 0.47898796, 0.82454437, ..., 1.        , 0.82301139,
        0.71674332],
       [0.75650301, 0.42830392, 0.86006469, ..., 0.82301139, 1.        ,
        0.81882193],
       [0.65661498, 0.39266756, 0.7783489 , ..., 0.71674332, 0.81882193,
        1.        ]])

In [25]:
from sklearn.neighbors import NearestNeighbors

In [26]:
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]
neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
neigh.fit(samples)

NearestNeighbors(n_neighbors=2, radius=0.4)

array([[2, 0]])